# Welcome to the qBraid presentation on techinques to embed classical images into quantum registers !

<img src="./_images/scheme.png">


Quantum computing (QC) is acknowledged to have significant speedups in solving intractable problems for classical computation by utilizing quantum mechanical phenmena such as entanglement and superposition. Current development of QCs have allowed for successful manipulation of approximately several dozen to a hundred noisy qubits, placing progress of various quantum architectures to be in the Noisy Intermediate Scale Quantum (NISQ) computing era. A promising area of research is Quantum Machine Learning (QML) which is known to provide atypical calculation patterns as well as relatively strong performances with low parameter usage compared to their classical counterparts.


Today, we'll be using qBraid to understand how to embed classical images into quantum registers, the current embedding implementations, and the current limitations and problems at hand. 

Agenda

- Getting Started with qBraid
- Classical implementation
- Quantum Embedding/Implementation
- qBraid SDK

## Getting Started on qBraid

### Step 1.
If you haven't already done so, please make a qBraid account and add the access code `EHNU6626` on the account.qbraid.com/account-details page. 


### Step 2.
Then click on the `Launch on qBraid` button in the README.md of this repository. The button will automatically clone the repository and take you to your *new* qBraid Lab integrated development environment. 

### Step 3.
Finally, install the qBraid-SDK environment via the qBraid Lab Environment Manager. On Lab you should see the `ENVS` icon on the right. The qBraid Lab Environment Manager is a robust package and virtual environment management system provided to qBraid end-users through a simple, intuitive graphical user interface. To expand the Environment Manager sidebar, click on Envs in the upper-right of the Lab console. My Environments are your currently installed environments. The qBraid Default environment and Microsoft Q# environment are installed by default.

Install environment
1. In the Environment Manager sidebar, click Add to view the environments available to install.

Choose the qBraid SDK, expand its panel, and click Install.

<img src="./_images/env_install.png">

3. Once the installation has started, the pannel is moved to the My Environments tab. Click Browse Environments to return to the My Environments tab and view its progress.



Browse Environments to return to the My Environments tab and view its progress.

<img src="./_images/env_installing.png">

4. When the installation is complete, the environment panel’s action button will switch from Installing… to Activate. Clicking Activate creates a new ipykernel, see Kernels for more.

<img src="./_images/kernel_activate.png">

To uninstall the environment, click on More, and then Uninstall. Learn more about qBraid Lab Environment Manager [here](https://qbraid-qbraid.readthedocs-hosted.com/en/stable/lab/environments.html#)

## Activate the qBraid SDK kernel
Under My Environments, choose the environment, and expand its pannel. Click Activate to activate the environment and create an associated ipykernel.

<img src="./_images/kernel_activate.png">

Switch notebook kernel
In the Launcher tab, under Notebooks, clicking on an ipykernel associated with an activated environment will automatically launch a Jupyter notebook (.ipynb file) using that kernel. In the upper-right of the newly created notebook, you can see which kernel is in use.

<img src="./_images/kernel_nb.png">

Clicking on the name of the current kernel, as circled above, will open the kernel selector, and allow you switch to any other active kernel.

<img src="./_images/kernel_switch.png">


Next we'll install tensorflow:

In [1]:
%pip install tensorflow
%pip install tensorboard
%pip install scikit-plot


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml
from qbraid import circuit_wrapper
from scikitplot.metrics import plot_confusion_matrix
import tensorflow as tf



%matplotlib inline
# CONSTANTS
NUM_EXAMPLES=500
os.environ['TENSORBOARD_BINARY'] = '/home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy/pyenv/bin/tensorboard'
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


# MNIST
The MNIST (Modified National Institute of Standards and Technology) database contains 70,000 28 x 28 images of handwritten digits from 0-9 and is seminal to machine learning. The MNIST handwritten dataset is the “Hello World” implementation for machine learning, and the dataset is used as a worldwide machine learning benchmark. 


## Starting off with a classical implementation
We will first load the data and apply a classical CNN (Convolutional Neural Network) to understand the mechanics of image classification.

### Loading the data
We load the data from tensorflow, a machine learning package developed by Google.

In [ ]:
# Normalize the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))

We one hot encode the categories into ten classes.

In [ ]:
y_train_onehot = tf.one_hot(y_train,10)
y_test_onehot = tf.one_hot(y_test,10)

Let's plot the images to see what we're going to be classifying and embedding into quantum circuits.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
    plt.xlabel(y_train[i])
plt.show()

### Run a CNN (convolutional neural network)
In the following cells we run a simple CNN which has 2 convolutional layers and will classify, on average, to 98%. A convolutional layer is an integral transform which detects certain features using a filter to pass over the image.
<img src="./_images/convolutionalfilter.gif">


You can learn more about classical CNNs here.

To gather data on our model per epoch, we will include tensorboard and the model checkpoint callback.

In [ ]:
# Tensorboard callback
%load_ext tensorboard

# Specify Folders
current_time = str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
train_log_dir = 'logs/tensorboard/' + current_time
test_log_dir = 'logs/tensorboard/test/' + current_time
path_for_checkpoint_callback = 'logs/summary/'+current_time

# Create callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(
                log_dir=train_log_dir, histogram_freq=1, profile_batch=3
            )

# Model Checkpoint callback
model_ckpt =  tf.keras.callbacks.ModelCheckpoint(
                path_for_checkpoint_callback, save_weights_only=True
            )

The classical model is a sequential CNN for 10 class multiclassification. The model contains  1,199,882 trainable parameters.

In [ ]:
def create_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(28,28,1)))
    model.add(tf.keras.layers.Conv2D(64, [3, 3], activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    return model


cmodel = create_classical_model()
cmodel.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.002),
               metrics=[tf.keras.metrics.CategoricalAccuracy()])

cmodel.summary()

In [ ]:
cmodel.fit(x_train,
          y_train_onehot,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test[:1000], y_test_onehot[:1000]),
          callbacks=[tensorboard_callback,model_ckpt])

cnn_results = cmodel.evaluate(x_test, y_test_onehot)

### Analysis of classical layers
The CNN model accomplishes the classification task incredibly well averaging 98% in performance. We can confirm the performance with the confusion matrix where the model predicted the hand written digits for all 10000 images in the test set. The state of the art classical image classification ML models can accomplish similar performances for incredibly complex images using customized feature extraction, trainable layers and other techniques. 

In [ ]:
y_pred = cmodel.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred, y_test)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
plot_confusion_matrix(y_test, y_pred, ax=ax)

Let's see how a quantum machine learning model stacks up.

# QML
In this era, Parameterized Quantum Circuits (PQC), where gates consist of fixed and trained unitary operators which perform rotations on qubits, dominate in the implementation of QML algorithms. More specifically, PQC based QML algorithms currently involve seminal machine learning datasets such as MNIST, fashion-MNIST, CIFAR-10 to understand the performance of various architectures such as TTN, MERA, and ladder-lik. For MNIST classification, several of the aforementioned circuits have produced comparable or outperforming results for binary classification compared to their classical counterparts. The performances of the most recent models achieve 95% to 97% for 5 vs 3 classification and are consistently outperforming fair classical models. The strong binary classification results and consistent improvements since Farhi et al's binary classification, which achieved 85\%, strongly suggesting that multi-classification is an exciting step in furthering understanding the expressivity of PQCs. Models such as those by Chalumuri et al. which achieves 92.10% on the IRIS dataset Bokhan et al. where their model classified 4 classes to acheive 85.14% and 90.03%, as well as Zeng et al. have shown exciting potential for QNN models in multiclassification.


## Quantum Computing and it's proposed benefits and limitations
Using a quantum computer for a classical machine learning is still a very new and active area of research. In fact, this notebook should illustrate to you *why* there needs to be immense progress in hardware and software until a quantum computer is suitable for a classical image classifiation task. I personally found MNIST to be my hello world to ML, and hearing that there are some CS background individuals I hope that this notebook can be a hellow world to QML. The notebook isn't inteded to be a fun and wonderfully entry point for those with an inclination for machine learning to appreciate the exciting current challenges in QML.


### Possible, theoretical, benefits:

- Improving time complextities of linear algebra subroutines (singular value decomposition) using kernel methods.

- Exponential time complexity speed ups 

- Overall there needs to be more understanding of how to frame the problem.

https://quantumcomputing.stackexchange.com/questions/13531/what-is-the-advantage-of-quantum-machine-learning-over-traditional-machine-learn

### Current challenges

These challenges are just some out of many which require further research

#### 1: embedding and decoding classical data using quantum circuits

During the NISQ era, there are severe limitations with how to encoding the images onto a quantum register for gate based implementations. For MNIST, the full image cannot be encoded into a quantum register; therefore, downsampling techniques are used to reduce the image size. In addition instead of implementing QRAM, embedding techniques such as angle and amplitude encodings are used to convert the classicla data to quantum data on the fly.



#### 2: there aren't enough qubits, existing qubits are prone to error, and QRAM is not available.
Current NISQ quantum computers are too noisy and have too few qubits for actually obtaining results. It's just as effective to use a noise model and a simulator to understand some of the quantum machine learning algorithms on hardware.

#### 3: cost of running circuits

The cost of running an actual QML task on a quantum computer can be incredibly expensive $3 ~ 4k (USD) as of 2022. This is most apparent on the IonQ device and the accessibility of the devices are supported by credit programs by various groups such as AWS, IBM, and of course qBraid!


## Preprocess the MNIST dataset for quantum computing
For this demo, we will only apply a classical preprocessing technique suitable to different embedding techniques. It must be noted that angle embedding will require distorting the image to a shape of 8 x 4 for this simple demo. The classical preprocessing can, in fact, be customized a lot more to maximize the effectiveness of the embedding technique. Our demo is purely intended to show the current techniques.

### Preprocess
The preprocess will encode the image using 8 qubits and various embedding techniques. 

Angle embeddingl:
1. Remove the 5 pixel border around the image which contains no data.
2. Resize the image to 8 x 4.


Aamplitude embedding:
1. Remove the 5 pixel border around the image which contains no data.
2. Resize the image to 16 x 16.

In [ ]:
# Crop and remove border from image
x_train_crop = np.array([x.reshape(28,28)[4:26,4:26] for x in x_train])
x_test_crop =np.array([x.reshape(28,28)[4:26,3:26] for x in x_test])

# Add channel dimension to dataset for resizing [ batch, height, width, channel]
x_train_crop = x_train_crop[..., np.newaxis]
x_test_crop = x_test_crop[..., np.newaxis]


# Resize image to fit on register for angle embedding
x_train_32 = tf.image.resize(x_train_crop, [8,8]).numpy()
x_test_32 = tf.image.resize(x_test_crop, [8,8]).numpy()


# Resize image to fit on register for amplitude embedding
x_train_256 = tf.image.resize(x_train_crop, [16,16]).numpy()
x_test_256 = tf.image.resize(x_test_crop, [16,16]).numpy()

# Confirm dataset is still correct batch and shape
print(f'The batch size is: {len(x_train_256)} images')
print(f'Image height and width (angle): {x_train_256[10].shape} \n')

# Confirm dataset is still correct batch and shape
print(f'The batch size is: {len(x_train_256)} images')
print(f'Image height and width (amplitude): {x_train_256[10].shape}')

In [ ]:
# Let's check the results of the preprocessing for amplitude embedding
plt.imshow(x_train_256[599])

In [ ]:
# Let's check the results of the preprocessing for angle embedding
plt.imshow(x_train_32[599])

### 3 vs 5? Caveats to the image preprocessing:
The downsampling can result in images that are so distorted that it may not be recognizable even by humans on what digit it represents. Let's just say that the preprocessing can produce some messy handwriting.

<img src="./_images/3vs5.png">

## Methods of embedding the image into a circuit

First we need to pre-process the data to ensure it can be embedded.

In [11]:
#constants and universal functions
WIRES=8
weight_shapes = {"weights": (1,8,3) }

#### Angle Embedding
Angle embedding encodes the image data as rotations applied to the qubit. To fit the data on 8 qubits we will encode each of the pixels as Pauli Z rotations. The rotations will be applied for every row of the image.

$$U_{\text{angle encoding}} = Z^{N_i} \mid n_i \rangle$$

<img src="./_images/angle_embedding.webp">
          

In [12]:
# we will develop the embedding circuit
dev = qml.device('default.qubit', wires=WIRES)
@qml.qnode(dev)
def angle_embedding(inputs):
    # We apply a Pauli Z rotation to incode every pixel in the 8 x 8
    for i in range(WIRES): 
        qml.AngleEmbedding(features=inputs[i].flatten(), wires=range(WIRES), rotation='Z')
    return qml.expval(qml.PauliZ(0))

In [13]:
shape = qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=8)
weights = np.random.random(size=shape)
print(weights.shape)
print(qml.draw(angle_embedding)(x_train_32[0]))

(1, 8, 3)
0: ─╭AngleEmbedding(M0)─╭AngleEmbedding(M1)─╭AngleEmbedding(M2)─╭AngleEmbedding(M3)
1: ─├AngleEmbedding(M0)─├AngleEmbedding(M1)─├AngleEmbedding(M2)─├AngleEmbedding(M3)
2: ─├AngleEmbedding(M0)─├AngleEmbedding(M1)─├AngleEmbedding(M2)─├AngleEmbedding(M3)
3: ─├AngleEmbedding(M0)─├AngleEmbedding(M1)─├AngleEmbedding(M2)─├AngleEmbedding(M3)
4: ─├AngleEmbedding(M0)─├AngleEmbedding(M1)─├AngleEmbedding(M2)─├AngleEmbedding(M3)
5: ─├AngleEmbedding(M0)─├AngleEmbedding(M1)─├AngleEmbedding(M2)─├AngleEmbedding(M3)
6: ─├AngleEmbedding(M0)─├AngleEmbedding(M1)─├AngleEmbedding(M2)─├AngleEmbedding(M3)
7: ─╰AngleEmbedding(M0)─╰AngleEmbedding(M1)─╰AngleEmbedding(M2)─╰AngleEmbedding(M3)

──╭AngleEmbedding(M4)─╭AngleEmbedding(M5)─╭AngleEmbedding(M6)─╭AngleEmbedding(M7)─┤  <Z>
──├AngleEmbedding(M4)─├AngleEmbedding(M5)─├AngleEmbedding(M6)─├AngleEmbedding(M7)─┤     
──├AngleEmbedding(M4)─├AngleEmbedding(M5)─├AngleEmbedding(M6)─├AngleEmbedding(M7)─┤     
──├AngleEmbedding(M4)─├AngleEmbedding(M5)─├AngleEm

In [ ]:
"""
Create quantum layer which only contains the embedding strongly entangling layers 
used in the circuit centric classifer by Shuld et al. and the AMM readout strategy by Zeng et al.
"""

def amm_strategy(wires): 
    """
    All-qubit Multi-observable Measurement (AMM) strategy applies pauli operators 
    X, Y, Z on all qubits to extract features from disentangled quantum state
    (Zeng et al.).
    """
    readout = []
    # Applies pauli
    for i in range(8):
        readout.append(qml.expval(qml.PauliX(i)))
        readout.append(qml.expval(qml.PauliY(i)))
        readout.append(qml.expval(qml.PauliZ(i)))            
    return  readout
    
    
@qml.qnode(dev)
def angle_layer(inputs,weights):
    """
    Quantum Angle embedding layer which will apply the embedding.
    """
    for i in range(8): 
        qml.AngleEmbedding(features=tf.reshape(inputs[0],[-1]), wires=range(8), rotation='Z')
    qml.templates.StronglyEntanglingLayers(weights, wires=range(8))
    return amm_strategy(wires)

In [ ]:
qlayer_angle = qml.qnn.KerasLayer(angle_layer, weight_shapes, output_dim=24)

In [ ]:
# Build the Keras model for angle embedding.
qmodel_angle = tf.keras.Sequential([
# The input is the data-circuit, encoded as a tf.string
    qlayer_angle,
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
qmodel_angle.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    optimizer=tf.keras.optimizers.Adam(0.02))
qmodel_angle.build(input_shape=[32,8,8,1])

In [ ]:
qmodel_angle.summary()

In [ ]:
qnn_angle_history = qmodel_angle.fit(x_train_32[:1000],
          y_train_onehot[:1000],
          batch_size=32,
          epochs=5,
          verbose=1,
          validation_data=(x_test_32[:1000], y_test_onehot[:1000]),
          callbacks=[tensorboard_callback,model_ckpt])

In [ ]:
qnn_angle_results = qmodel_angle.evaluate(x_test_32, y_test_onehot)

In [ ]:
y_pred_angle = qmodel_angle.predict(x_test_256)
y_pred_angle = np.argmax(y_pred, axis=1)
print(y_pred_amplitude, y_test)

In [ ]:
fig, ax = plt.subplots(figsize=(16,12))
plot_confusion_matrix(y_test, y_pred_angle, ax=ax)

#### Amplitude Embedding (wave function embedding)
With amplitude embedding we will encode $2^n$ features where $n$ represents the number of qubits. In simple terms, the amplitude is the height of a wave. In this kind of embedding the data points are transformed into amplitudes of the quantum state. 


For example the classical data $[1,2,3,4]$ can be normalized and encoded on two qubits as so $$\mid \psi \rangle = \frac{1}{\sqrt{30}} (\mid 00 \rangle + 2 \mid 01 \rangle + 3 \mid 10 \rangle + 4 \mid 00 \rangle)$$

In [ ]:
# we will develop the embedding circuit
dev = qml.device('default.qubit', wires=WIRES)
@qml.qnode(dev)
def amplitude_embedding(inputs):
    qml.AmplitudeEmbedding(features=inputs, wires=range(WIRES), normalize=True)
    return qml.expval(qml.PauliZ(0))

We can verify that our amplitude properly embeds an image into the circuit. The circuit encodes 256 pixels on 8 registers (wires) and returns a Z basis measurement of the first wire. 

##### What does this embedding strategy look like?
By displaying the block sphere we see that the circuit is 


In [42]:
@qml.qnode(dev)
def amplitude_layer(inputs,weights):
    """
    Quantum amplitude encoding layer which will apply the embedding.
    """
    qml.AmplitudeEmbedding(features=inputs, wires=range(WIRES), normalize=True)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(WIRES))
    return amm_strategy(wires)

In [43]:
shape = qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=WIRES)
weights = np.random.random(size=shape)
print(weights.shape)
print(qml.draw(amplitude_layer)(x_train_256[0].flatten(), weights))

(1, 8, 3)


NameError: name 'amm_strategy' is not defined

In [44]:
qlayer_amplitude = qml.qnn.KerasLayer(amplitude_layer, weight_shapes, output_dim=24)

The model we will use will only use the embedding circuit and the strongly entangling layers to verify the performance. We will keep the quantum layers to be the same for each of the embedding processes.

In [ ]:
# Build the Keras model for amplitude embedding.
qmodel_amplitude = tf.keras.Sequential([
# The input is the data-circuit, encoded as a tf.string
    tf.keras.layers.Flatten(),
    qlayer_amplitude,
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
qmodel_amplitude.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    optimizer=tf.keras.optimizers.Adam(0.02))
qmodel_amplitude.build(input_shape=[32,16,16,1])

In [ ]:
qmodel_amplitude.summary()

In [ ]:
qnn_history_amplitude = qmodel_amplitude.fit(x_train_256[:1000],
          y_train_onehot[:1000],
          batch_size=32,
          epochs=5,
          verbose=1,
          validation_data=(x_test_256[:1000], y_test_onehot[:1000]),
          callbacks=[tensorboard_callback,model_ckpt])

In [ ]:
qnn_amplitude_results = qmodel_amplitude.evaluate(x_test_256, y_test_onehot)

In [ ]:
y_pred_amplitude = qmodel_amplitude.predict(x_test_256)
y_pred_amplitude = np.argmax(y_pred, axis=1)
print(y_pred_amplitude, y_test)

In [ ]:
fig, ax = plt.subplots(figsize=(16,12))
plot_confusion_matrix(y_test, y_pred_amplitude, ax=ax)

#### Full model results for various papers

<img src="./_images/10_class.png" width="450px">

There are plenty of amazing examples of work that are achieving improvements in training PQC for MNIST image classification. Here's one


<img src="./_images/training_final.png">

The sectional angle embedding technique which allows for minimal
downsampling of classical MNIST image dataset using 10 qubits and averaging
to 85.32% over 10 trials is demonstrated. In doing so, the HQNN results in com-
parable performances of 89.09% by Zeng et al.[1] for 10 class multi-classification.
While optimistic performances were achieved, sectioned angle embedding allows
for a desired number of sections embedded on single qubit gates suggesting further improvements to be explored to reduce the qubit count and improve the image granularity.

### Running a fair model


In [ ]:
# Build the Keras model.
fair_model = tf.keras.Sequential([
# The input is the data-circuit, encoded as a tf.string
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
fair_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    optimizer=tf.keras.optimizers.Adam(0.02))
fair_model.build(input_shape=[32,16,16,1])

In [ ]:
fair_model.summary()

In [ ]:
fair_model_history = fair_model.fit(x_train_256[:1000],
          y_train_onehot[:1000],
          batch_size=32,
          epochs=5,
          verbose=1,
          validation_data=(x_test_256[:1000], y_test_onehot[:1000]),
          callbacks=[tensorboard_callback,model_ckpt])

In [ ]:
fair_results = fair_model.evaluate(x_test_256, y_test_onehot)

In [ ]:
y_fair_pred = fair_model.predict(x_test_256)
y_fair_pred = np.argmax(y_fair_pred, axis=1)
print(y_test, y_fair_pred)

In [ ]:
fig, ax = plt.subplots(figsize=(16,12))
plot_confusion_matrix(y_test, y_fair_pred, ax=ax)


# Using the qBraid SDK to transpile and run on multiple devices

<img src="./_images/qbraid_logo_white_large.png" width=100px>
The qBraid SDK is a Python toolkit for building and executing quantum programs.


Features:

- Unified quantum frontend interface. Transpile quantum circuits between supported packages. Leverage the capabilities of multiple frontends through simple, consistent protocols.
- Build once, target many. Create quantum programs using your preferred circuit-building package, and execute on any backend that interfaces with a supported frontend.
- Benchmark, compare, interpret results. Built-in compatible post-processing enables comparing results between runs and across backends.


In [53]:
""" 
We will create the angle embedding as a QML Tape which records
and stores variational quantum programs.
"""
with qml.tape.QuantumTape() as tape:
    # Apply angle embedding to 1 image
    angle_embedding(x_train_32[0])
    
# Apply qbraid sdk circuit wrapper
pennylane_circuit = circuit_wrapper(tape)

Once we have wrapped the circuit, we will now transpile the circuit to various other frameworks and see what the circuit looks like!

In [54]:
from qbraid import SUPPORTED_PROGRAM_TYPES
for _, (k, v) in enumerate(SUPPORTED_PROGRAM_TYPES.items()):
    print(k,v)

cirq Circuit
pyquil Program
qiskit QuantumCircuit
braket Circuit
pennylane QuantumTape


In [55]:
braket_circuit = pennylane_circuit.transpile("cirq")
print(braket_circuit)

q_0: ───Rz(0)─────────Rz(0)─────────Rz(0)─────────Rz(0)────────Rz(0)────────Rz(0)─────────Rz(0)─────────Rz(-1.61π)────M('c_0')───

q_1: ───Rz(0)─────────Rz(0.522π)────Rz(0)─────────Rz(0)────────Rz(0)────────Rz(0)─────────Rz(-1.15π)────Rz(-0.671π)──────────────

q_2: ───Rz(0)─────────Rz(0.532π)────Rz(1.84π)─────Rz(0)────────Rz(0)────────Rz(0)─────────Rz(-0.441π)───Rz(0.649π)───────────────

q_3: ───Rz(1.01π)─────Rz(-0.835π)───Rz(-0.459π)───Rz(-1.42π)───Rz(0.224π)───Rz(0.144π)────Rz(-0.107π)───Rz(0)────────────────────

q_4: ───Rz(-0.666π)───Rz(1.72π)─────Rz(0)─────────Rz(1.96π)────Rz(0.955π)───Rz(-0.614π)───Rz(-0.842π)───Rz(0)────────────────────

q_5: ───Rz(-1.2π)─────Rz(-1.72π)────Rz(0)─────────Rz(0.373π)───Rz(1.89π)────Rz(0.049π)────Rz(0)─────────Rz(0)────────────────────

q_6: ───Rz(-1.81π)────Rz(1.92π)─────Rz(0)─────────Rz(0)────────Rz(0)────────Rz(0)─────────Rz(0)─────────Rz(0)────────────────────

q_7: ───Rz(0)─────────Rz(0)─────────Rz(0)─────────Rz(0)────────Rz(0)────────Rz(0)──

In [56]:
cirq_circuit = circuit_wrapper(braket_circuit).transpile("cirq")
print(cirq_circuit)

q_0: ───Rz(0)─────────Rz(0)─────────Rz(0)─────────Rz(0)────────Rz(0)────────Rz(0)─────────Rz(0)─────────Rz(-1.61π)────M('c_0')───

q_1: ───Rz(0)─────────Rz(0.522π)────Rz(0)─────────Rz(0)────────Rz(0)────────Rz(0)─────────Rz(-1.15π)────Rz(-0.671π)──────────────

q_2: ───Rz(0)─────────Rz(0.532π)────Rz(1.84π)─────Rz(0)────────Rz(0)────────Rz(0)─────────Rz(-0.441π)───Rz(0.649π)───────────────

q_3: ───Rz(1.01π)─────Rz(-0.835π)───Rz(-0.459π)───Rz(-1.42π)───Rz(0.224π)───Rz(0.144π)────Rz(-0.107π)───Rz(0)────────────────────

q_4: ───Rz(-0.666π)───Rz(1.72π)─────Rz(0)─────────Rz(1.96π)────Rz(0.955π)───Rz(-0.614π)───Rz(-0.842π)───Rz(0)────────────────────

q_5: ───Rz(-1.2π)─────Rz(-1.72π)────Rz(0)─────────Rz(0.373π)───Rz(1.89π)────Rz(0.049π)────Rz(0)─────────Rz(0)────────────────────

q_6: ───Rz(-1.81π)────Rz(1.92π)─────Rz(0)─────────Rz(0)────────Rz(0)────────Rz(0)─────────Rz(0)─────────Rz(0)────────────────────

q_7: ───Rz(0)─────────Rz(0)─────────Rz(0)─────────Rz(0)────────Rz(0)────────Rz(0)──

In [57]:
qiskit_circuit = circuit_wrapper(cirq_circuit).transpile("qiskit")
qiskit_circuit.draw()

┌───────┐     ┌───────┐     ┌───────┐     ┌───────┐   »
    q_0: ──┤ Rz(0) ├─────┤ Rz(0) ├─────┤ Rz(0) ├─────┤ Rz(0) ├───»
         ┌─┴───────┴─┐ ┌─┴───────┴──┐  ├───────┤     ├───────┤   »
    q_1: ┤ Rz(220.5) ├─┤ Rz(18.609) ├──┤ Rz(0) ├─────┤ Rz(0) ├───»
         ├───────────┴┐├────────────┤  ├───────┤   ┌─┴───────┴──┐»
    q_2: ┤ Rz(71.641) ├┤ Rz(32.297) ├──┤ Rz(0) ├───┤ Rz(1.1719) ├»
         ├────────────┤├────────────┤  ├───────┤   ├────────────┤»
    q_3: ┤ Rz(98.437) ├┤ Rz(244.17) ├──┤ Rz(0) ├───┤ Rz(131.81) ├»
         ├───────────┬┘├───────────┬┘┌─┴───────┴──┐├────────────┤»
    q_4: ┤ Rz(15.75) ├─┤ Rz(248.7) ├─┤ Rz(111.66) ├┤ Rz(221.73) ├»
         └─┬───────┬─┘ └┬─────────┬┘ ├────────────┤└─┬───────┬──┘»
    q_5: ──┤ Rz(0) ├────┤ Rz(253) ├──┤ Rz(56.031) ├──┤ Rz(0) ├───»
           ├───────┤   ┌┴─────────┴─┐└─┬───────┬──┘  ├───────┤   »
    q_6: ──┤ Rz(0) ├───┤ Rz(152.44) ├──┤ Rz(0) ├─────┤ Rz(0) ├───»
           ├───────┤   └─┬───────┬──┘  ├───────┤     ├───────┤   »
    q_7: ──┤ Rz(0) ├─────┤ Rz(0) ├─────┤ Rz(0) ├─────┤ Rz(0) ├───»
           └───────┘     └───────┘     └───────┘     └───────┘   »
m_c_0: 1/════════════════════════════════════════════════════════»
                                                                 »
«            ┌───────┐     ┌───────┐     ┌───────┐     ┌───────┐      
«    q_0: ───┤ Rz(0) ├─────┤ Rz(0) ├─────┤ Rz(0) ├─────┤ Rz(0) ├──────
«            ├───────┤     ├───────┤     ├───────┤     ├───────┤      
«    q_1: ───┤ Rz(0) ├─────┤ Rz(0) ├─────┤ Rz(0) ├─────┤ Rz(0) ├──────
«          ┌─┴───────┴──┐┌─┴───────┴──┐  ├───────┤     ├───────┤      
«    q_2: ─┤ Rz(194.42) ├┤ Rz(213.78) ├──┤ Rz(0) ├─────┤ Rz(0) ├──────
«          ├────────────┤├────────────┤┌─┴───────┴──┐  ├───────┤      
«    q_3: ─┤ Rz(103.53) ├┤ Rz(224.27) ├┤ Rz(60.188) ├──┤ Rz(0) ├──────
«         ┌┴────────────┤├────────────┤├────────────┤  ├───────┤      
«    q_4: ┤ Rz(0.70312) ├┤ Rz(100.98) ├┤ Rz(225.86) ├──┤ Rz(0) ├──────
«         └──┬───────┬──┘└─┬───────┬──┘├────────────┤┌─┴───────┴──┐   
«    q_5: ───┤ Rz(0) ├─────┤ Rz(0) ├───┤ Rz(237.38) ├┤ Rz(77.438) ├───
«            ├───────┤     ├───────┤   ├────────────┤├────────────┤   
«    q_6: ───┤ Rz(0) ├─────┤ Rz(0) ├───┤ Rz(84.359) ├┤ Rz(198.95) ├───
«            ├───────┤     ├───────┤   └─┬───────┬──┘├────────────┤┌─┐
«    q_7: ───┤ Rz(0) ├─────┤ Rz(0) ├─────┤ Rz(0) ├───┤ Rz(208.58) ├┤M├
«            └───────┘     └───────┘     └───────┘   └────────────┘└╥┘
«m_c_0: 1/══════════════════════════════════════════════════════════╩═
«                                                                   0

In [58]:
pyquil_circuit = circuit_wrapper(qiskit_circuit).transpile("pyquil")
print(pyquil_circuit)

DECLARE m0 BIT[1]
RZ(0) 0
RZ(220.5) 1
RZ(71.640625) 2
RZ(98.4375) 3
RZ(15.75) 4
RZ(0) 5
RZ(0) 6
RZ(0) 7
RZ(0) 0
RZ(18.609375) 1
RZ(32.296875) 2
RZ(244.17188000000002) 3
RZ(248.70312) 4
RZ(252.99999999999997) 5
RZ(152.4375) 6
RZ(0) 7
RZ(0) 0
RZ(0) 1
RZ(0) 2
RZ(0) 3
RZ(111.65625000000001) 4
RZ(56.03125000000001) 5
RZ(0) 6
RZ(0) 7
RZ(0) 0
RZ(0) 1
RZ(1.171875) 2
RZ(131.8125) 3
RZ(221.73438) 4
RZ(0) 5
RZ(0) 6
RZ(0) 7
RZ(0) 0
RZ(0) 1
RZ(194.42188) 2
RZ(103.53124999999999) 3
RZ(0.703125) 4
RZ(0) 5
RZ(0) 6
RZ(0) 7
RZ(0) 0
RZ(0) 1
RZ(213.78125000000003) 2
RZ(224.26562) 3
RZ(100.98438000000002) 4
RZ(0) 5
RZ(0) 6
RZ(0) 7
RZ(0) 0
RZ(0) 1
RZ(0) 2
RZ(60.1875) 3
RZ(225.85938) 4
RZ(237.37499999999997) 5
RZ(84.359375) 6
RZ(0) 7
RZ(0) 0
RZ(0) 1
RZ(0) 2
RZ(0) 3
RZ(0) 4
RZ(77.4375) 5
RZ(198.95312) 6
RZ(208.57812) 7
MEASURE 7 m0[0]



In [ ]:
pyquil_circuit = circuit_wrapper(qiskit_circuit).transpile("pyquil")
print(pyquil_circuit)

# Citations
1. Zeng Y, Wang H, He J, Huang Q, Chang S. "A Multi-Classification Hybrid Quan-
tum Neural Network Using an All-Qubit Multi-Observable Measurement Strategy."
Entropy. 2022; 24(3):394. https://doi.org/10.3390/e24030394
2. Farhi, Edward, and Hartmut Neven. "Classification with quantum neural networks
on near term processors." arXiv preprint arXiv:1802.06002 (2018).https://doi.
org/10.48550/arXiv.1802.06002
3. Bokhan, Denis, Alena S. Mastiukova, Aleksey S. Boev, Dmitrii N. Trubnikov, and
Aleksey K. Fedorov. "Multiclass classification using quantum convolutional neural
networks with hybrid quantum-classical learning." arXiv preprint arXiv:2203.15368
(2022).
4. Chalumuri, A., Kune, R. Manoj, B.S. A hybrid classical-quantum approach for
multi-class classification. Quantum Inf Process 20, 119 (2021).https://doi.org/
10.1007/s11128-021-03029-9
5. Feynman, R.P. Simulating physics with computers. Int J Theor Phys 21, 467–488
(1982). https://doi.org/10.1007/BF02650179
6. Biamonte, Jacob, Peter Wittek, Nicola Pancotti, Patrick Rebentrost, Nathan Wiebe,
and Seth Lloyd. "Quantum machine learning." Nature 549, no. 7671 (2017): 195-202.
7. Huggins, William, Piyush Patil, Bradley Mitchell, K. Birgitta Whaley, and E. Miles
Stoudenmire. "Towards quantum machine learning with tensor networks." Quantum
Science and technology 4, no. 2 (2019): 024001.
8. Oh, Seunghyeok, Jaeho Choi, and Joongheon Kim. "A tutorial on quantum convolu-
tional neural networks (QCNN)." In 2020 International Conference on Information
and Communication Technology Convergence (ICTC), pp. 236-239. IEEE, 2020.
9. Jiang, W., Xiong, J. Shi, Y. A co-design framework of neural networks and quantum
circuits towards quantum advantage. Nat Commun 12, 579 (2021). https://doi.
org/10.1038/s41467-020-20729-5
10. Hellstern, G.: Analysis of a hybrid quantum network for classification tasks. IET
Quant. Comm. 2( 4), 153– 159 (2021). https://doi.org/10.1049/qtc2.12017
11. Arunachalam, Srinivasan, and Ronald de Wolf. "Guest column: A survey of quan-
tum learning theory." ACM Sigact News 48, no. 2 (2017): 41-67.
12. Schuld, Maria, and Nathan Killoran. "Is quantum advantage the right
goal for quantum machine learning?." arXiv preprint arXiv:2203.01340 (2022).
